# compute pH, omega_aragonite, omega_calcite -> EULERIAN OUTPUT
# use mocsy routines
# NOTE: to be able to call the fortran package, use python3!!!
# TRY TO SPEED THINGS UP!

In [ ]:
# some useful links:

# getting started with mocsy: http://ocmip5.ipsl.jussieu.fr/mocsy/fort.html
# getting started with using mocsy in python: http://ocmip5.ipsl.jussieu.fr/mocsy/pyth.html
# mocsy code: https://github.com/jamesorr/mocsy
# example notebooks: https://github.com/jamesorr/mocsy/tree/master/notebooks
# paper about mocsy: https://gmd.copernicus.org/articles/8/485/2015/gmd-8-485-2015.pdf 


In [ ]:
#!jupyter nbconvert --to script save_carbonate_chemistry_on_floats.ipynb

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
#sys.path.append("python_gsw_py3/") # add GSW to search path -> python3 version
sys.path.append("mocsy/")
#sys.path.append('/home/ollie/jhauck/py_fesom/modules/')
sys.path.append('/global/homes/c/cnissen/scripts_Ollie/python_gsw_py3/gsw/gibbs/')
import os
#import pyfesom2 as pf
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
import mocsy
from gsw import p_from_z # get pressure from z and lat
import timeit
from numba import njit
import xarray as xr
from tqdm import tqdm
import time

In [2]:
#-----
# get some general infor about mocsy
#-----

#print(mocsy.__version__)
#print(mocsy.__file__)
#dir(mocsy)
#print(mocsy.mvars.__doc__)
#print(mocsy.mbuffesm.__doc__)


In [3]:
#-----
# load float output, calculate carbonate chemistry at each location, store in netcdf file
#-----
# what fields from model do I need?
#    tos, sos, alk (surface only), dic (surface only), sio2 (surface only), din (surface only), slp
#

#----
# some settings
#----

# path to float data
path = '/global/cfs/cdirs/m4003/maltrud/6year/monthlyEulerianAverages/'

year_list = ['0055','0056','0057','0058','0059','0060']
print(year_list)

# variables to save as netcdf 
# -> Note: if you want to save different variables, 
#          the code piece for writing these fields into a netcdf file needs to be adjusted
#vari_list = ['pH', 'co2', 'hco3', 'co3', 'gammadic', 'gammaalk', 'rf']
vari_list = ['pH', 'OmegaA','OmegaC']#,'pco2','co3']

pressure_unit_conversion = 101325 # convert from Pa to atm
# particle_atmPressure, N m-2 = Pa

save_netcdf = True
savepath    = '/global/cfs/cdirs/m4003/cnissen/floats/6year/'
if not os.path.exists(savepath):
    print('Created '+savepath)
    os.makedirs(savepath)

#---
# read mesh info
#---
rad_to_deg = 180.0/np.pi

path_mesh = '/global/cfs/cdirs/m4003/maltrud/'
meshID = 'EC30to60E2r2'
meshfile = xr. open_dataset(path_mesh+'ocean.'+meshID+'.210210.nc')
#print(meshfile)

lon  = meshfile['lonCell'].values*rad_to_deg
lat  = meshfile['latCell'].values*rad_to_deg
topo = meshfile['bottomDepth'].values
area = meshfile['areaCell'].values
zlevs            = meshfile['refBottomDepth'].values
layerThickness   = meshfile['layerThickness'].values
restingThickness = meshfile['restingThickness'].values

print(len(lon),'nodes in mesh')
print(topo.shape)
print(area.shape)
print('Min/Max lon:',np.min(lon),np.max(lon))
print('Min/Max lat:',np.min(lat),np.max(lat))
print('layerThickness.shape:',layerThickness.shape)
print('restingThickness.shape:',restingThickness.shape)

meshfile.close()


['0055', '0056', '0057', '0058', '0059', '0060']
236853 nodes in mesh
(236853,)
(236853,)
Min/Max lon: 0.0007300572350528742 359.997672445938
Min/Max lat: -78.53259417674468 89.94461290099375
layerThickness.shape: (1, 236853, 60)
restingThickness.shape: (236853, 60)


In [16]:
#---
# loop over years, load data, compute carbonate chemistry, store variables
#---
# tos, sos, alk (surface only), dic (surface only), sio2 (surface only), din (surface only), slp
# 

save_netcdf = True

factor_conc = 1./1000. # to convert concentrations from mmol m-3 to mol m-3

for yy in range(1,len(year_list)):
    print('Load year '+year_list[yy])
    file1 = 'monthlyAverageEulerianFields.year'+year_list[yy]+'.nc' 
    data = xr. open_dataset(path+file1)
    temp = data['timeMonthly_avg_activeTracers_temperature']#.values # time x depth x floats
    salt = data['timeMonthly_avg_activeTracers_salinity']#.values
    alk  = data['timeMonthly_avg_ecosysTracers_ALK']#.values
    dic  = data['timeMonthly_avg_ecosysTracers_DIC']#.values
    sio  = data['timeMonthly_avg_ecosysTracers_SiO3']#.values
    po4  = data['timeMonthly_avg_ecosysTracers_PO4']#.values
    slp  = data['timeMonthly_avg_atmosphericPressure']#.values # time x floats   
    data.close()
    
    # get number of grid cells
    num_cells = len(data['nCells'].values)
    # get time
    num_time = len(data['record'].values)
    # get number of depth levels
    num_depth = len(data['nVertLevels'].values)
    
    if save_netcdf: 
        fv = -9999
        for vv in range(0,len(vari_list)):
            netcdf_name = vari_list[vv]+'_monthlyAverageEulerianFields.year'+year_list[yy]+'.nc' 
            if not os.path.exists(savepath+netcdf_name):
                print('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
                w_nc_fid.mocsy_dir    = '/global/homes/c/cnissen/scripts/mocsy'
                # create dimension & variable
                w_nc_fid.createDimension('record', num_time)  
                w_nc_fid.createDimension('nCells', num_cells)  
                w_nc_fid.createDimension('nVertLevels', num_depth) 
                w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('record','nCells','nVertLevels'),fill_value=fv)
                w_nc_var1.data_source = path+file1
                #w_nc_var1.unit = 'degrees E'
                w_nc_fid.close()
                del netcdf_name
                
    start = time.time()                                
    for tt in tqdm(range(0,num_time)):
        for dd in range(0,num_depth):
            
            dic2  = np.squeeze(dic[tt,0,:,dd].values)
            # timeMonthly_avg_ecosysTracers_DIC(record, Time, nCells, nVertLevels)
            
            # only continue if at least one entry exists
            if np.max(dic2)>0: 
                
                temp2 = np.squeeze(temp[tt,0,:,dd].values)
                salt2 = np.squeeze(salt[tt,0,:,dd].values)
                alk2  = np.squeeze(alk[tt,0,:,dd].values)
                sio2  = np.squeeze(sio[tt,0,:,dd].values)
                po42  = np.squeeze(po4[tt,0,:,dd].values)
                slp2  = np.squeeze(slp[tt,0,:].values)
                
                ind_av = np.where(dic2>-1)[0]

                if zlevs[dd]<0:
                    pressure = p_from_z(zlevs[dd],lat[ind_av])
                elif zlevs[dd]>=0:
                    pressure = p_from_z(-1*zlevs[dd],lat[ind_av]) # to make sure that pressure is >0
                #print('pressure',pressure)

                pH = np.nan*np.zeros(num_cells)
                pco2 = np.nan*np.zeros(num_cells)
                fco2 = np.nan*np.zeros(num_cells)
                co2 = np.nan*np.zeros(num_cells)
                hco3 = np.nan*np.zeros(num_cells)
                co3 = np.nan*np.zeros(num_cells)
                OmegaA = np.nan*np.zeros(num_cells)
                OmegaC = np.nan*np.zeros(num_cells)
                BetaD = np.nan*np.zeros(num_cells)
                DENis = np.nan*np.zeros(num_cells)
                p = np.nan*np.zeros(num_cells)
                Tis = np.nan*np.zeros(num_cells)

                #gammadic = np.nan*np.zeros(num_time)
                #betadic = np.nan*np.zeros(num_time)
                #omegadic = np.nan*np.zeros(num_time)
                #gammaalk = np.nan*np.zeros(num_time)
                #betaalk = np.nan*np.zeros(num_time)
                #omegaalk = np.nan*np.zeros(num_time)
                #rf = np.nan*np.zeros(num_time)

                # make sure to convert slp from Pa to atm; units of nutrients etc. should be mol m-3
                pH[ind_av],pco2[ind_av],fco2[ind_av],co2[ind_av],\
                    hco3[ind_av],co3[ind_av],OmegaA[ind_av],OmegaC[ind_av],\
                    BetaD[ind_av],DENis[ind_av],p[ind_av],Tis[ind_av] = mocsy.mvars(temp2[ind_av],
                            salt2[ind_av], alk2[ind_av]*factor_conc, dic2[ind_av]*factor_conc,\
                            sio2[ind_av]*factor_conc, po42[ind_av]*factor_conc, 
                            slp2[ind_av]*pressure_unit_conversion, pressure, lat[ind_av],optcon='mol/m3', optt='Tpot', 
                            optp='db', optb="u74", optk1k2='l', optkf="dg", optgas="Pinsitu")
                #print('Done with computing carbonate chemistry')
                
                #gammadic[ind_av],betadic[ind_av],omegadic[ind_av],\
                #        gammaalk[ind_av],betaalk[ind_av],omegaalk[ind_av],rf[ind_av] = mocsy.mbuffesm(temp[:,dd][ind_av], salt[:,dd][ind_av],
                #                                        alk[:,dd][ind_av]*factor_conc, dic[:,dd][ind_av]*factor_conc,
                #                                        sio[:,dd][ind_av]*factor_conc, po4[:,dd][ind_av]*factor_conc,
                #                                        slp[ind_av], pressure, lat[ind_av],
                #                                        optcon='mol/m3', optt='Tpot',optp='db',
                #                                        optb="u74", optk1k2='l', optkf="dg", optgas="Pinsitu")

                # to store: pH, co2, hco3, co3, gammadic, gammaalk, rf
                if save_netcdf: 
                    for vv in range(0,len(vari_list)):
                        netcdf_name = vari_list[vv]+'_monthlyAverageEulerianFields.year'+year_list[yy]+'.nc' 
                        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                        if vari_list[vv] in ['pH']:
                            pH[np.isnan(pH)]=fv
                            w_nc_fid.variables[vari_list[vv]][tt,:,dd] = pH
                        elif vari_list[vv] in ['OmegaA']:
                            OmegaA[np.isnan(OmegaA)]=fv
                            w_nc_fid.variables[vari_list[vv]][tt,:,dd] = OmegaA
                        elif vari_list[vv] in ['OmegaC']:
                            OmegaC[np.isnan(OmegaC)]=fv
                            w_nc_fid.variables[vari_list[vv]][tt,:,dd] = OmegaC
                        w_nc_fid.close()  
                        del netcdf_name

            del slp2,dic2,alk2,sio2,po42,temp2,salt2
        
    end = time.time()
    print(end - start)
    
print('done')


Load year 0056
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/pH_monthlyAverageEulerianFields.year0056.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaA_monthlyAverageEulerianFields.year0056.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaC_monthlyAverageEulerianFields.year0056.nc


100%|██████████| 12/12 [06:45<00:00, 33.79s/it]


405.50501251220703
Load year 0057
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/pH_monthlyAverageEulerianFields.year0057.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaA_monthlyAverageEulerianFields.year0057.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaC_monthlyAverageEulerianFields.year0057.nc


100%|██████████| 12/12 [06:39<00:00, 33.30s/it]


399.62709379196167
Load year 0058
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/pH_monthlyAverageEulerianFields.year0058.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaA_monthlyAverageEulerianFields.year0058.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaC_monthlyAverageEulerianFields.year0058.nc


...

411.6273868083954
Load year 0059
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/pH_monthlyAverageEulerianFields.year0059.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaA_monthlyAverageEulerianFields.year0059.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaC_monthlyAverageEulerianFields.year0059.nc


100%|██████████| 12/12 [06:41<00:00, 33.45s/it]


401.4023835659027
Load year 0060
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/pH_monthlyAverageEulerianFields.year0060.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaA_monthlyAverageEulerianFields.year0060.nc
Create file /global/cfs/cdirs/m4003/cnissen/floats/6year/OmegaC_monthlyAverageEulerianFields.year0060.nc


100%|██████████| 12/12 [06:51<00:00, 34.30s/it]

411.5998623371124
done
